In [75]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from scipy import stats as ss
import seaborn as sns
from sklearn import metrics
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import KFold

from pygam import LinearGAM, s, l

from sklearn import linear_model

import xgboost

import statsmodels.api as sm

In [76]:
# Đọc dữ liệu từ file
dataX = pd.read_csv("X_train.csv")
dataY = pd.read_csv("Y_train.csv")
dataX.drop(['id'], axis=1, inplace=True, errors='ignore')
dataY.drop(['id'], axis=1, inplace=True, errors='ignore')

# Tạo dataframe có cả features và outcome
data = dataX.copy()
data.insert(22, "price", dataY['price'], True) 
data.head(5)

,manufacturer,model,transmission,color,odometer,year,engineFuel,engineType,engineCapacity,bodyType,...,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,price
0,Hyundai,i40,automatic,red,48000,2014,gasoline,gasoline,2.0,sedan,...,True,False,False,False,True,False,False,True,True,15500.0
1,Mitsubishi,Carisma,mechanical,green,320000,2000,diesel,diesel,1.9,hatchback,...,True,False,False,False,False,False,False,False,False,2800.0
2,Volkswagen,T5,mechanical,white,164000,2011,diesel,diesel,2.0,minibus,...,True,False,True,False,False,True,True,True,False,16700.0
3,Volkswagen,T4 Multivan,mechanical,blue,385672,1998,diesel,diesel,2.5,minibus,...,True,True,False,False,False,False,False,True,True,11000.0
4,Toyota,Camry,automatic,black,215652,2005,gasoline,gasoline,2.4,sedan,...,False,False,False,False,False,False,False,False,True,6800.0


### Handle Missing Values

In [77]:
NaNRows = data[data.isna().any(axis=1)]
dataX.drop(index=NaNRows.index, errors='ignore', inplace=True)
dataY.drop(index=NaNRows.index, errors='ignore', inplace=True)
data.drop(index=NaNRows.index, errors='ignore', inplace=True)
print(len(dataX.index))
print(len(dataY.index))
print(len(data.index))

29991
29991
29991


### Delete duplicates

In [78]:
duplicateRows = dataX[dataX.duplicated(keep=False)]
index1=duplicateRows.index
duplicateRows = data[data.duplicated(keep=False)]
index2=duplicateRows.index

duplicateRows = data[data.duplicated(keep='first')]
dataX.drop(index=duplicateRows.index, errors='ignore', inplace=True)
dataY.drop(index=duplicateRows.index, errors='ignore', inplace=True)
data.drop(index=duplicateRows.index, errors='ignore', inplace=True)
print(len(dataX.index))
print(len(dataY.index))
print(len(data.index))

index=list(set(index1)-set(index2))
dupData = data.loc[index].sort_values(by=['manufacturer', 'model'])
index = list(dupData.index)
i = 0
while i < len(index) - 1:
    newPrice = (dupData.loc[index[i]].price + dupData.loc[index[i+1]].price) / 2
    
    # adjust 'price' values
    dataY.at[index[i], 'price'] = newPrice 
    data.at[index[i], 'price'] = newPrice
    
    # drop duplicate records
    dataX.drop(index=index[i+1], errors='ignore', inplace=True)
    dataY.drop(index=index[i+1], errors='ignore', inplace=True)
    data.drop(index=index[i+1], errors='ignore', inplace=True)

    i = i + 2
print(len(dataX.index))
print(len(dataY.index))
print(len(data.index))

29966
29966
29966
29949
29949
29949


### Features Selection

In [79]:
# Features selected: ['manufacturer', 'transmission', 'color', 'odometer', 'year,
#       'engineType', 'engineCapacity', 'bodyType', 'drivetrain',
#       'photos', 'feature_0', 'feature_1', 'feature_2', 'feature_3',
#       'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8',
#       'feature_9']
# Features eliminated: ['model', 'engineFuel'] 
dataX.drop(columns=['model', 'engineFuel'], axis=1, inplace=True, errors='ignore')
print(len(dataX.keys()))

20


### Normalize / Standardize

In [80]:
# Get column names first
numericFeatures = ['odometer','year','engineCapacity','photos']
# Create the Scaler object
scaler = preprocessing.StandardScaler()

# Standardize dataX
# Fit your data on the scaler object
scaled_df = scaler.fit_transform(dataX[numericFeatures])
scaled_df = pd.DataFrame(scaled_df, index=dataX[numericFeatures].index, columns=numericFeatures)
# Update dataX 
dataX.update(scaled_df)
dataX[numericFeatures]

,odometer,year,engineCapacity,photos
0,-1.559269,1.446670,-0.081713,-1.419866
1,0.510816,-0.416370,-0.232640,0.537986
2,-0.676439,1.047448,-0.081713,0.701140
3,1.010619,-0.682518,0.672921,-1.419866
4,-0.283336,0.249002,0.521994,0.374832
...,...,...,...,...
29995,-0.006706,0.648225,0.521994,-0.604094
29996,0.282497,-0.815593,-0.383566,-0.114631
29997,-0.021927,-1.347890,-0.081713,-0.440940
29998,0.107454,-0.150221,-0.534493,-0.604094


### Dummmy

In [81]:
dataX = pd.get_dummies(dataX[dataX.keys()], drop_first = True)
print(len(dataX.keys()))
print(dataX.keys())

89
Index(['odometer', 'year', 'engineCapacity', 'photos', 'feature_0',
       'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5',
       'feature_6', 'feature_7', 'feature_8', 'feature_9',
       'manufacturer_Alfa Romeo', 'manufacturer_Audi', 'manufacturer_BMW',
       'manufacturer_Buick', 'manufacturer_Cadillac', 'manufacturer_Chery',
       'manufacturer_Chevrolet', 'manufacturer_Chrysler',
       'manufacturer_Citroen', 'manufacturer_Dacia', 'manufacturer_Daewoo',
       'manufacturer_Dodge', 'manufacturer_Fiat', 'manufacturer_Ford',
       'manufacturer_Geely', 'manufacturer_Great Wall', 'manufacturer_Honda',
       'manufacturer_Hyundai', 'manufacturer_Infiniti', 'manufacturer_Iveco',
       'manufacturer_Jaguar', 'manufacturer_Jeep', 'manufacturer_Kia',
       'manufacturer_LADA', 'manufacturer_Lancia', 'manufacturer_Land Rover',
       'manufacturer_Lexus', 'manufacturer_Lifan', 'manufacturer_Lincoln',
       'manufacturer_Mazda', 'manufacturer_Mercedes-Benz', 'ma

### Model Building

In [82]:
# Ordinary Least Squares
kf = KFold(n_splits = 10, shuffle = True)
n = kf.get_n_splits(dataX)
avg_rmse = 0.0
for train_index, test_index in kf.split(dataX):
    x_train = dataX.iloc[train_index]
    x_test = dataX.iloc[test_index]
    y_train = dataY.iloc[train_index]
    y_test = dataY.iloc[test_index]
    regressor = linear_model.LinearRegression(n_jobs=20)
    regressor.fit(x_train, y_train)
    y_pred = regressor.predict(x_test)
    rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
    print(rmse)
    avg_rmse += rmse / n
print('Average RMSE: ', avg_rmse)

2894.885922907907
3273.8262856585234
2889.834522460454
3050.973799108674
3039.6066003393717
2937.1457168348265
2912.601477930931
3149.8945032830457
2866.1421206454925
3023.0659958962956
Average RMSE:  3003.7976945065516


In [83]:
# polynomial # GAM
kf = KFold(n_splits = 10, shuffle = True)
n = kf.get_n_splits(dataX)
avg_rmse = 0.0
for train_index, test_index in kf.split(dataX):
    x_train = dataX.iloc[train_index]
    x_test = dataX.iloc[test_index]
    y_train = dataY.iloc[train_index]
    y_test = dataY.iloc[test_index]
    gam = LinearGAM(s(0, n_splines=12) + s(1, n_splines=12) + s(2, n_splines=12) + s(3, n_splines=12) + l(4) + l(5) + l(6) + l(7) + l(8) + l(9) + 
                    l(10) + l(11) + l(12) + l(13) + l(14) + l(15) + l(16) + l(17) + l(18) + l(19) +
                    l(20) + l(21) + l(22) + l(23) + l(24) + l(25) + l(26) + l(27) + l(28) + l(29) + 
                    l(30) + l(31) + l(32) + l(33) + l(34) + l(35) + l(36) + l(37) + l(38) + l(39) + 
                    l(40) + l(41) + l(42) + l(43) + l(44) + l(45) + l(46) + l(47) + l(48) + l(49) + 
                    l(50) + l(51) + l(52) + l(53) + l(54) + l(55) + l(56) + l(57) + l(58) + l(59) +
                    l(60) + l(61) + l(62) + l(63) + l(64) + l(65) + l(66) + l(67) + l(68) + l(69) +
                    l(70) + l(71) + l(72) + l(73) + l(74) + l(75) + l(76) + l(77) + l(78) + l(79) +
                    l(80) + l(81) + l(82) + l(83) + l(84) + l(85) + l(86) + l(87) + l(88)).fit(x_train,y_train)
    y_pred = gam.predict(x_test)
    rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
    print(rmse)
    avg_rmse += rmse / n
print('Average RMSE: ', avg_rmse)

2422.368131045709
2643.407658795647
2700.013455438851
2560.1362166397844
2543.705329467894
2573.1137613102974
2555.839803586336
2770.770180421158
2689.6690671780193
2626.896009729708
Average RMSE:  2608.5919613613405


In [ ]:
# Extreme Boosting Regression
kf = KFold(n_splits = 10, shuffle = True)
n = kf.get_n_splits(dataX)
avg_rmse = 0.0
for train_index, test_index in kf.split(dataX):
    x_train = dataX.iloc[train_index]
    x_test = dataX.iloc[test_index]
    y_train = dataY.iloc[train_index]
    y_test = dataY.iloc[test_index]
    xgb = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)
    xgb.fit(x_train,y_train)
    y_pred = xgb.predict(x_test)
    rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
    print(rmse)
    avg_rmse += rmse / n
print('Average RMSE: ', avg_rmse)

[13:38:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
1834.2775032622394
[13:39:14] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
1790.6543739292135
[13:39:39] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
1779.365115999707
[13:40:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
1654.0628127028215
[13:40:27] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
1831.0632355125774
[13:40:51] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
1770.9922600863074
[13:41:15] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
1896.09348463421
[13:41:39] WARNING: src/objective/regression_obj.cu:152: r